In [1]:
import os
import sys
directory_path = os.path.abspath('/notebooks/')
if directory_path not in sys.path:
    sys.path.append(directory_path)

import pandas as pd
import numpy as np

In [13]:
policies_raw = pd.read_csv('../data/policies.csv')
cols_to_drop = ['zip', 'county_name', 'Agent_cd']
policies_raw = policies_raw.drop(cols_to_drop, axis=1)

df = policies_raw

before = df.shape[0]
df = df.dropna()
after = df.shape[0]
print(before)

49162


In [11]:
from hash_map import Mapping

dropped_policy_ids = Mapping()

# verify that counts of is_missing is correct
df = policies_raw
before = df.shape[0]
df = df.dropna()
after = df.shape[0]
print(before - after)

# TODO: fix hash mapping dropped policy ids
df_missing_vals = policies_raw.dropna(how='all').dropna(how='all', axis=1)
is_missing = df_missing_vals.isnull().any(axis=1)
print(np.unique(is_missing, return_counts=True))
# print(len(is_missing.filter(like="")))
# for item in range(len(is_missing)):
#     if is_missing[item]: dropped_policy_ids.hash(policies_raw['policy_id'][item])


16986
(array([False,  True]), array([32176, 16986], dtype=int64))
